In [16]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    BertTokenizer, 
    BertForSequenceClassification, 
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"当前设备: {device.upper()}")
if device == "cuda":
    print(f"显卡型号: {torch.cuda.get_device_name(0)}")

当前设备: CUDA
显卡型号: NVIDIA GeForce RTX 3060 Laptop GPU


In [3]:
# 2. 数据准备
dataset = load_dataset("holistic-ai/EMGSD")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(examples):
    tokenized = tokenizer(examples["text"], truncation=True, max_length=128)
    
    # 标签处理逻辑
    labels = []
    for l in examples["label"]:
        if isinstance(l, str) and (l.startswith("stereotype") or l == "related"):
            labels.append(1)
        else:
            labels.append(0)
    tokenized["labels"] = labels
    return tokenized

print("正在处理数据...")
tokenized_datasets = dataset.map(
    preprocess_function, 
    batched=True,
    remove_columns=dataset["train"].column_names
)
print("数据处理完成！")

E:\PY\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


正在处理数据...


Map:   0%|          | 0/45760 [00:00<?, ? examples/s]

Map:   0%|          | 0/11441 [00:00<?, ? examples/s]

数据处理完成！


In [10]:
# 3. 模型初始化
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels=2
).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# 4. 定义指标
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    
    # [cite_start]Macro F1 是论文的核心指标 [cite: 139]
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc, 
        'f1_macro': f1
    }

In [20]:
# 5. 训练参数设置
training_args = TrainingArguments(
    output_dir="./hearts_bert_finetuned",
    
    # --- 学习率与 Epoch ---
    learning_rate=2e-5,
    num_train_epochs=6,              
    weight_decay=0.1,                   # 正则化防过拟合
    
    # --- 显存与速度优化 ---
    per_device_train_batch_size=8,       # 小 Batch 防止爆显存
    gradient_accumulation_steps=4,       # 累积梯度，等效 Batch Size = 32
    per_device_eval_batch_size=8,
    fp16=(device == "cuda"),             # 混合精度加速
    
    # --- 评估策略 ---
    evaluation_strategy="epoch",         # 每个 epoch 评测
    save_strategy="epoch",               # 每个 epoch 保存
    load_best_model_at_end=True,         # 训练结束回滚到最佳模型
    
    #metric_for_best_model="f1_macro",    # 依据 F1 判断

    metric_for_best_model="eval_loss",    # 根据Loss判断
    
    #save_total_limit=2,                  # 只保留2个模型节省空间
    
    logging_steps=10,
    report_to="none"
)

In [21]:
# 6. 初始化 Trainer
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    
    # Dynamic Padding
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),

    compute_metrics=compute_metrics,
    
    # 连续 3 次没提升就停止
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

print(" 机器要开始学习了...")
trainer.train()

E:\PY\Lib\site-packages\accelerate\accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


 机器要开始学习了...


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.069700,0.944857,0.856044,0.841626
2,0.032800,0.854025,0.858317,0.841283
3,0.042900,1.003955,0.853072,0.840871
4,0.028100,1.076295,0.859977,0.845401


Checkpoint destination directory ./hearts_bert_finetuned\checkpoint-1430 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./hearts_bert_finetuned\checkpoint-2860 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./hearts_bert_finetuned\checkpoint-4290 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./hearts_bert_finetuned\checkpoint-5720 already exists and is non-empty.Saving will proceed but saved results may be invalid.


KeyboardInterrupt: 

In [ ]:
# 7. 保存最佳模型
print(" 正在评估...")
metrics = trainer.evaluate()
print(f" 最终 F1: {metrics['eval_f1_macro']:.4f}")

trainer.save_model("./final_best_model")
print(" 模型已保存！")